# Basic scraping for Guardian discussions u

In [1]:
! ./.g


[master 01b461f] web
 1 file changed, 2373 insertions(+), 1017 deletions(-)
 rewrite guardian comments scraper.ipynb (61%)
remote: Support for password authentication was removed on August 13, 2021. Please use a personal access token instead.
remote: Please see https://github.blog/2020-12-15-token-authentication-requirements-for-git-operations/ for more information.
fatal: unable to access 'https://github.com/macbuse/GUARDIAN.git/': The requested URL returned error: 403


In [59]:
with open('guardian comments scraper.ipynb','r') as fp:
    nb = fp.read()

In [60]:
import re

pp = re.compile("(https://www.the.*?)'")
lks = pp.findall(nb)

with open('links.txt','a') as fp:
    fp.write('\n'.join(lks))

In [3]:
import re 
import requests
import pickle
import json

In [9]:
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/19/ditch-the-algorithm-generation-students-a-levels-politics'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/21/joe-biden-democrat-convention-nice-guy-trump-fight'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/23/the-cruel-exams-algorithm-has-laid-bare-the-unfairness-at-the-heart-of-our-schools'
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/01/us-mainstream-right-vigilante-terror'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/31/politicians-covid-19-statistics-statisticians'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'
article_url = 'https://www.theguardian.com/commentisfree/2021/may/10/retail-politics-movements-polarising-projects-promise'

In [15]:
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/29/rival-scientists-lockdowns-scientific-covid-19'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/08/johnson-and-sturgeon-will-need-a-better-philosophy-than-precaution-to-rid-us-of-covid'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/10/continual-local-lockdowns-answer-covid-control'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/11/the-rebel-scientists-cause-would-be-more-persuasive-if-it-werent-so-half-baked'

article_url = 'https://www.theguardian.com/commentisfree/2021/apr/03/undermining-the-astrazeneca-jab-is-a-dangerous-act-of-political-folly#comment-148440261'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/09/boris-johnson-brexit-belfast-violence-eu-good-friday-agreement'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/02/culture-war-government-race-report'
article_url = 'https://www.theguardian.com/commentisfree/2021/mar/18/britain-sunny-vaccine-optimism-april-delivery-delay'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/15/britain-harness-social-sciences-covid-pandemic-deprivation'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/16/government-race-report-evidence#comment-148662066'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/20/britain-falling-birthrate-covid-pandemic-conservatives-removed-support-for-parents'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/21/bmi-eating-disorder-pandemic-treatment'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/30/government-delays-covid-public-inquiry-health-crisis'
article_url = 'https://www.theguardian.com/commentisfree/2021/may/07/humiliation-hartlepool-smalltown-detective-di-starmer'
article_url ='https://www.theguardian.com/commentisfree/2021/aug/06/uk-government-experts-vaccinating-under-16s'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/01/there-is-a-case-for-vaccine-passports-but-ministers-are-failing-to-make-it'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/09/america-is-flying-blind-when-it-comes-to-the-delta-variant'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/10/covid-vaccines-britain-scientific-solution-jabs'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/13/denial-anti-vaxxers-climate-sceptics'
article_url = 'https://www.theguardian.com/commentisfree/2018/apr/11/bbc-andrew-neil-media-politics'

article_url = 'https://www.theguardian.com/commentisfree/2021/aug/15/vaccine-passports-are-less-a-threat-to-liberty-than-a-mark-of-solidarity'
article_url = 'https://www.theguardian.com/theobserver/commentisfree/2021/aug/14/will-we-reach-herd-immunity-for-the-new-coronavirus'

In [16]:

key = my_keys.keys['guardian']

NameError: name 'my_keys' is not defined

In [ ]:
#scrape the page and get the short url
# 15/8/21  this failed and gave /p/temp
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)
uu = r.text
print(short_url)

#the short url allows us to access the discussion
# 15/8/21 api.nextgen.guardianapps.co.uk/discussion/ failed on 
# https://discussion.theguardian.com/discussion-api/discussion/p/tfmxg
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 50
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn,'wb') as fp:
    pickle.dump(json_data,fp)

/p/tgxqd
getting comments page no. 1


In [14]:
uu


'<!doctype html>\n        <html lang="en">\n            <head>\n\t\t\t    \n<!--\n\n                                    GGGGGGGGG\n                           GGGGGGGGGGGGGGGGGGGGGGGGGG\n                       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n                    GGGGGGGGGGGGGGGGG      GG   GGGGGGGGGGGGG\n                  GGGGGGGGGGGG        GGGGGGGGG      GGGGGGGGGG\n                GGGGGGGGGGG         GGGGGGGGGGGGG       GGGGGGGGG\n              GGGGGGGGGG          GGGGGGGGGGGGGGGGG     GGGGGGGGGGG\n             GGGGGGGGG           GGGGGGGGGGGGGGGGGGG    GGGGGGGGGGGG\n            GGGGGGGGG           GGGGGGGGGGGGGGGGGGGGGG  GGGGGGGGGGGGG\n           GGGGGGGGG            GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n           GGGGGGGG             GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n          GGGGGGGG              GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n          GGGGGGGG              GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n          GGGGGGGG              GGGGGGGGGGGG              G

## remember each page of comments has been stored separately

In [16]:
dd = json.loads(json_data[0])
len(json_data)

11

In [17]:
xx = json.loads(json_data[0])
xx.keys()

dict_keys(['postedCommentHtml', 'commentsHtml', 'commentCount', 'refreshStatus', 'lastPage', 'paginationHtml'])

In [136]:
print( xx['commentsHtml'][200:700])

 class="d-comment
            d-comment--top-level"
           id="comment-144422422"
           data-comment-id="144422422"
           data-comment-author="Hushing"
           data-comment-replies="0"
           data-comment-timestamp="2020-10-11T14:25:06.000+01:00"
           data-comment-author-id="12407494"
           data-comment-highlighted="false"
           itemscope itemtype="http://schema.org/Comment">

    <div class="d-comment__inner d-comment__inner--top-level">
        <div class="


In [16]:
pa = re.compile('data-comment-author="(.*?)"')
pid = re.compile('data-comment-author-id="(\d+)')
pcid = re.compile('\sid="comment-(.*?)"')
pp = re.compile('<li class="d-comment(.*?)>',re.DOTALL)

auths = []
md = []
for xx in json_data:
    hh = json.loads(xx)['commentsHtml']
    auths.extend(list( zip(pa.findall(hh), 
                           pid.findall(hh)
                          )))
    md.extend(pp.findall(hh))

auths = list(set(auths))
auths.sort(key=lambda x : x[0].lower())


In [102]:
#print( '\n'.join([x for x in md if 'pinkie' in x.lower()]))

https://discussion.theguardian.com/comment-permalink/144375739

In [8]:
auths = dict(auths)

In [9]:
comments_url

'https://api.nextgen.guardianapps.co.uk/discussion//p/hvpq8.json'

In [12]:

from bs4 import BeautifulSoup

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  
            soup.findAll('div', {'class' : "d-comment__body"}) ]
    
    metadata = soup.findAll('span', {'class' : "d-comment__author"})
    
    
    auth = [ x.text.upper().strip() for x in metadata]
    auth_id = [ x.find('a')['href'] for x in metadata]
    #this was a pain to do really
    #the problem is blocked comments
    comment_id = [ x['id'][len('comment-'):] for x in soup.findAll('li')]
 
    
    ss = '##{}\nhttps://discussion.theguardian.com/comment-permalink/{}\n{}\n\n{}'
    return [ ss.format(a,b,d,c.strip())  
                   for a,b,d,c in zip(auth, comment_id, auth_id, raw)]
     
all_texts = [ html2comments(page) for page in json_data[:] ]



from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

# Random data hacking

In [6]:
A = [x.split('\n')[0] for x in all_comments]
from collections import Counter
AN = Counter(A)
sorted([(x,y) for x,y in AN.items() if y > 4], key=lambda x : -x[1]) 

[('##PAULOMAROC', 16),
 ('##ENGLISHATHENA', 15),
 ('##SFH765', 13),
 ('##KWAKWA', 11),
 ('##BEAUTIFULBLACKBIRD', 11),
 ('##THEREALCMDRGRAVY', 10),
 ('##PADDYTHEGREEK', 10),
 ('##STILLCARING', 10),
 ('##RATHERBEINOZ', 9),
 ('##MEREMORTAL', 9),
 ('##ME2015COM', 8),
 ('##FREEDOMOFSPEECG', 8),
 ('##SEVENQ', 7),
 ('##PROPERTYPETE79', 7),
 ('##TONDA99', 6),
 ('##VTIMAN', 6),
 ('##SNAFU5001', 6),
 ('##CHEESEWOMBAT', 6),
 ('##HFLASHMAN', 6),
 ('##ABETTERWORLDAWAITS', 6),
 ('##KEENY123', 5),
 ('##ANTICORBYNISTA', 5),
 ('##SKYMAGNET', 5),
 ('##VERAANTHONY', 5),
 ('##ERIKFREDERIKSEN', 5),
 ('##DENSHER', 5),
 ('##HEATHENVEGAN', 5),
 ('##MINTAKA', 5)]

In [8]:
! ./.g


[master 9889fa4] web
 3 files changed, 1017 insertions(+), 1281 deletions(-)
 rewrite guardian comments scraper.ipynb (76%)
 create mode 100644 te5tn.pkl
 create mode 100644 tedzk.pkl
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 570.67 KiB | 750.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 1 local object.
To https://github.com/macbuse/GUARDIAN.git
   51fb340..9889fa4  master -> master


In [7]:
print(all_comments[0])

##SIGMA79
https://discussion.theguardian.com/comment-permalink/151126119
https://profile.theguardian.com/user/id/17326520

This comment was removed by a moderator because it didn't abide by our
                            community standards.
                            Replies may also be deleted. For more detail see our FAQs.


In [13]:
all_comments.sort(key=lambda x : -len(x) )
print( '\n\n'.join(all_comments[:20]) )

##PUSHK1N
https://discussion.theguardian.com/comment-permalink/114618840
https://profile.theguardian.com/user/id/1611394

Paxman is and was a professional interviewer who was capable of separating his personal views and agenda from his Night job on Newsnight where he took his brief as holding Politicians of all blends to account. 
His most famous interview is probably that of Michael Howard asking whether or not he threatened Derek Lewis ,then Prison Boss of an arms length executive agency, with over ruling a decision he had made .
I think he asked him 17 times with a stony face " did you threaten to overrule him".
It made Michael Howard look well and truly the twerp he was, I think sometime later Anne Widdicombe further did for him by saying there was something of the night about Michael Howard.
Though he still went on to become Tory Leader, and I cannot remember what the fuss was all about.
I am not aware of any interview that Andrew Neil has ever done which was anything like the ico

In [18]:
#https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

import nltk
#nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gregmcshane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
a = '''My views haven't changed. Leaving the EU will be of no benefit to anyone except those who hold money and power along with certain corporates and a smidgen of medium to small businesses.
How else can it be when you are shutting shop to the most developed nations, the richest nations in the world, who are either in the EU or have market agreements with the EU. Even the emerging markets of China and India along with other developing economies are aspiring to develop trading agreements with the EU, the second largest economy in the world. Any deal with the UK will be secondary and will depend upon their agreement with the EU.
Boris Johnson once said "f**k business". I take this as meaning there is something in it for them and they don't really care if businesses find it difficulA
t to cope.
I can see ministers and conservative MPs queuing to fill their pockets by acting as advisors for potential trade deals whilst the rest of the country are left struggling trying to rebuild their lives, especially following the pandemic. The same ministers will probably be looking to lower all kinds of standards from food and welfare to safety at work to improve their opportunities.
When it comes to brinkmanship, I can only view this as getting away with whatever they can. I do not believe it is anything to do with negotiating an agreement that will be to the benefit of the UK or the EU as a whole, I don't believe our ministers are capable of doing that.'''

In [24]:
sid.polarity_scores(a)
sid.sentiment_valence(a)

TypeError: sentiment_valence() missing 4 required positional arguments: 'sentitext', 'item', 'i', and 'sentiments'

In [9]:
target = 'decolon'
target = 'science'
target = 'denial'


xx = [x for x in all_comments if target.lower() in x.lower() and  len(x) > 20]
yy = [x for x in all_comments if '#pin' in x.lower() and 2000 > len(x) > 20]

#print('\n\n'.join( ['\n\n'.join(x) for x in zip(xx,yy)])  )

#print( '\n\n'.join(xx).replace(target, ) )
pxx = re.compile(target, re.I)
print( pxx.sub('**' + target.upper() + '**', 
              '\n\n'.join(xx)) )
               
#print( '\n\n'.join(xx) )


##SMELLTHECOFFEE101
https://discussion.theguardian.com/comment-permalink/151247108
https://profile.theguardian.com/user/id/10204723

Are you in **DENIAL**? Because it’s not just anti-vaxxers and climate sceptics

Thats true, lots of people in **DENIAL** about lots of things.

To accept the facts about climate science without changing the way we live is also to deny reality

I accept the facts about climate science. I have a couple of problems with changing the way I live. The first is I simply cant afford to. 
Like many in the ;rich' UK I am desperately poor. I own the home I live in (in a poor area of the UK). BUT. ...... 
I drive an 11 year old diesel car, because I can't afford to replace it. My bathroom and kitchen are 25 years old, because I can't afford to replace them. My, very limited collection of clothes, probably the newest item is 4 years old. Replace my ancient gas boiler, you say? How? Eat less meat? I can't afford meat already, maybe 500g minced steak a week as a treat? 

In [20]:
(26/31) / (74/69)

0.7820401046207497

In [64]:
23**2

529

## 

In [21]:
84 + 127

211

In [24]:
84/211/.31

1.2842073077511085

In [25]:
127/211/.69

0.8723126588364586

In [26]:
1.28/.87

1.471264367816092